In [11]:
import os
from dotenv import load_dotenv
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [12]:
@tool
def multiply_tool(x: int, y: int) -> int:
    """Multiply two numbers."""
    return x * y

In [13]:
multiply_tool.args

{'x': {'title': 'X', 'type': 'integer'},
 'y': {'title': 'Y', 'type': 'integer'}}

In [14]:
LLM = ChatGroq(groq_api_key=os.environ.get("GROQ_API_KEY"), model="openai/gpt-oss-120b", temperature=1.2)
LLM_with_tool = LLM.bind_tools([multiply_tool])

In [15]:
LLM_with_tool.invoke('Hi how are you')

AIMessage(content='Hey there! I’m doing great, thanks for asking. How can I help you today?', additional_kwargs={'reasoning_content': 'User says "Hi how are you". It\'s a greeting, simple. Should respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 125, 'total_tokens': 172, 'completion_time': 0.09536686, 'prompt_time': 0.021061509, 'queue_time': 0.04361735, 'total_time': 0.116428369}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_91af62a853', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--89a4e6ca-dd3f-413e-bb8b-327ed6474060-0', usage_metadata={'input_tokens': 125, 'output_tokens': 47, 'total_tokens': 172})

In [16]:
query = HumanMessage('can you multiply 3 with 1000')
messages = [query]

In [17]:
result = LLM_with_tool.invoke(messages)

In [18]:
messages.append(result)
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': "The user asks to multiply 3 with 1000. Simple. I can compute 3000. I could also use the provided function multiply_tool. Let's do that.", 'tool_calls': [{'id': 'fc_fab5f9e8-ae2f-47a3-8f8c-2494e1926408', 'function': {'arguments': '{"x":3,"y":1000}', 'name': 'multiply_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 130, 'total_tokens': 201, 'completion_time': 0.141138727, 'prompt_time': 0.010046498, 'queue_time': 0.043520827, 'total_time': 0.151185225}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_4913bcd64f', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--18e6f0f8-e6db-47b3-b30c-b40ce2d15b91-0', tool_calls=[{'name': 'multiply_tool', 'args': {'x': 3, 'y': 1000}, 'id': 'fc_fab5f9e8-ae2f-47a3-8f8c-2494e1926408', 

In [19]:
tool_result = multiply_tool.invoke(result.tool_calls[0])

In [20]:
messages.append(tool_result)

In [21]:
LLM_with_tool.invoke(messages).content

'The product of 3 and 1000 is **3000**.'